<a href="https://colab.research.google.com/github/naufalhawari/gemastik-data-minik-esteh/blob/main/esteh-juara-ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Dataset

## Installing Library Needs

In [1]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Libraries

In [3]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max.columns", 50)

## Retrieve and Preprocess Dataset

In [4]:
# file_path = "/content/drive/MyDrive/dataset/esteh-dataset.xlsx"
file_path = "https://raw.githubusercontent.com/naufalhawari/dataset-collection/main/esteh-dataset-1728.csv"

# sheet names if data retrieved from excel
# sheet_names = [
#     "jogja_smart_service",
#     "depok_single_window",
# ]

In [5]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(teks):
  stemmerFactory = StemmerFactory().create_stemmer()

  return stemmerFactory.stem(teks)


def create_dataset(file_path, sheet_names = None) :
  

  # reading from excel
  # dataset = pd.DataFrame()
  # for sheet_name in sheet_names:
  #   dataset = pd.concat([dataset, pd.read_excel(file_path, sheet_name = sheet_name)],ignore_index = True)

  # reading from csv through github
  dataset = pd.read_csv(file_path)
  
  dataset = dataset.loc[~dataset.duplicated()].copy() # removing duplicated record
  dataset = dataset.copy().dropna() # dropping records if there is any missing values
  dataset["kritik"] = dataset[["keluhan", "saran"]].max(axis = 1)
  dataset = dataset.drop(["saran", "keluhan"], axis = 1)
  dataset["ulasan"] = dataset["ulasan"].str.lower()
  dataset["ulasan"] = dataset["ulasan"].apply(lambda x: stemming(x))

  return dataset

In [6]:
# df = create_dataset(file_path)
df = pd.read_csv(file_path)

In [7]:
df.head(10)

,ulasan,apresiasi,kritik
0,aplikasi bagus buat lapor ini itu cuma saya ma...,1,1
1,tidak bisa ke menu jaki vaksin ini kenapa sih ...,0,1
2,hello jaki saya lapor aplikasi jakarta aman ap...,0,1
3,enak nya pake aplikasi jaki klo kita mau lapor...,1,0
4,gak pernah bagi daftar dadak kalo mau ke tebet...,0,1
5,sy beri 4 bintang biar makin semangat it jaki ...,1,1
6,aplikasi apa coba di guna tidk bisa sedang di ...,0,1
7,system baik terus mau daftar vaksin tidak bisa,0,1
8,nik sdh verifikasi tapi utk login daftar vaksi...,0,1
9,daftar sunggu ribet lbih gampang daftar akun b...,0,1


In [8]:
df.duplicated().sum()

6

In [9]:
len(df)

1691

## Splitting Dataset

In [10]:
from sklearn.model_selection import train_test_split

X = df.ulasan
y = df.drop("ulasan", axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = df.apresiasi)

# RNN Model

## Build Text Tokenizer

In [11]:
VOCAB_SIZE = 1200
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(X_train)

In [12]:
len(encoder.get_vocabulary())

1200

## Creating Model Architecture

In [13]:
lstm = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype = "string"),
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=100,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='sigmoid'),
])

bi_lstm = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype = "string"),
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=100,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='sigmoid'),
])

stacked_bi_lstm = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype = "string"),
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=100,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='sigmoid'),
])

gru = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype = "string"),
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=100,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.GRU(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='sigmoid'),
])

bi_gru= tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype = "string"),
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=100,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='sigmoid'),
])

stacked_bi_gru = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype = "string"),
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=100,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='sigmoid'),
])

## Model Compiling

In [14]:
lstm.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00003),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy(),
)


bi_lstm.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00003),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy(),
)



stacked_bi_lstm.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00003),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy(),
)



gru.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00003),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy(),
)



bi_gru.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00003),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy(),
)


stacked_bi_gru.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00003),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy(),
)


## Model Training

In [15]:
lstm.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    validation_data = (X_test, y_test),
    epochs = 25,
)

Epoch 1/25
85/85 [==============================] - 29s 183ms/step - loss: 0.6914 - binary_accuracy: 0.6087 - val_loss: 0.6894 - val_binary_accuracy: 0.6799
Epoch 2/25
85/85 [==============================] - 7s 81ms/step - loss: 0.6865 - binary_accuracy: 0.6886 - val_loss: 0.6838 - val_binary_accuracy: 0.6726
Epoch 3/25
85/85 [==============================] - 5s 60ms/step - loss: 0.6774 - binary_accuracy: 0.6927 - val_loss: 0.6713 - val_binary_accuracy: 0.6711
Epoch 4/25
85/85 [==============================] - 3s 31ms/step - loss: 0.6434 - binary_accuracy: 0.6930 - val_loss: 0.5966 - val_binary_accuracy: 0.6711
Epoch 5/25
85/85 [==============================] - 2s 19ms/step - loss: 0.5568 - binary_accuracy: 0.6930 - val_loss: 0.5485 - val_binary_accuracy: 0.6711
Epoch 6/25
85/85 [==============================] - 3s 30ms/step - loss: 0.5365 - binary_accuracy: 0.6960 - val_loss: 0.5350 - val_binary_accuracy: 0.6844
Epoch 7/25
85/85 [==============================] - 2s 23ms/step - l

In [16]:
bi_lstm.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    validation_data = (X_test, y_test),
    epochs = 25,
)

Epoch 1/25
85/85 [==============================] - 23s 158ms/step - loss: 0.6905 - binary_accuracy: 0.6379 - val_loss: 0.6883 - val_binary_accuracy: 0.6681
Epoch 2/25
85/85 [==============================] - 9s 100ms/step - loss: 0.6829 - binary_accuracy: 0.7004 - val_loss: 0.6789 - val_binary_accuracy: 0.6711
Epoch 3/25
85/85 [==============================] - 4s 43ms/step - loss: 0.6679 - binary_accuracy: 0.6930 - val_loss: 0.6566 - val_binary_accuracy: 0.6711
Epoch 4/25
85/85 [==============================] - 3s 37ms/step - loss: 0.6090 - binary_accuracy: 0.6934 - val_loss: 0.5652 - val_binary_accuracy: 0.6711
Epoch 5/25
85/85 [==============================] - 2s 26ms/step - loss: 0.5554 - binary_accuracy: 0.6934 - val_loss: 0.5480 - val_binary_accuracy: 0.6711
Epoch 6/25
85/85 [==============================] - 3s 41ms/step - loss: 0.5345 - binary_accuracy: 0.6990 - val_loss: 0.5307 - val_binary_accuracy: 0.6799
Epoch 7/25
85/85 [==============================] - 3s 30ms/step - 

In [17]:
stacked_bi_lstm.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    validation_data = (X_test, y_test),
    epochs = 25,
)

Epoch 1/25
85/85 [==============================] - 36s 216ms/step - loss: 0.6902 - binary_accuracy: 0.6165 - val_loss: 0.6864 - val_binary_accuracy: 0.6696
Epoch 2/25
85/85 [==============================] - 7s 77ms/step - loss: 0.6748 - binary_accuracy: 0.6723 - val_loss: 0.6618 - val_binary_accuracy: 0.6711
Epoch 3/25
85/85 [==============================] - 5s 63ms/step - loss: 0.6090 - binary_accuracy: 0.6675 - val_loss: 0.5790 - val_binary_accuracy: 0.6711
Epoch 4/25
85/85 [==============================] - 5s 61ms/step - loss: 0.5490 - binary_accuracy: 0.6882 - val_loss: 0.5546 - val_binary_accuracy: 0.6711
Epoch 5/25
85/85 [==============================] - 4s 46ms/step - loss: 0.5231 - binary_accuracy: 0.6905 - val_loss: 0.5313 - val_binary_accuracy: 0.6726
Epoch 6/25
85/85 [==============================] - 4s 42ms/step - loss: 0.5002 - binary_accuracy: 0.7001 - val_loss: 0.5021 - val_binary_accuracy: 0.7006
Epoch 7/25
85/85 [==============================] - 5s 56ms/step - l

In [18]:
gru.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    validation_data = (X_test, y_test),
    epochs = 25,
)

Epoch 1/25
85/85 [==============================] - 17s 149ms/step - loss: 0.6917 - binary_accuracy: 0.5695 - val_loss: 0.6897 - val_binary_accuracy: 0.6490
Epoch 2/25
85/85 [==============================] - 5s 61ms/step - loss: 0.6868 - binary_accuracy: 0.6686 - val_loss: 0.6850 - val_binary_accuracy: 0.6740
Epoch 3/25
85/85 [==============================] - 4s 50ms/step - loss: 0.6800 - binary_accuracy: 0.6930 - val_loss: 0.6782 - val_binary_accuracy: 0.6711
Epoch 4/25
85/85 [==============================] - 2s 27ms/step - loss: 0.6715 - binary_accuracy: 0.6927 - val_loss: 0.6685 - val_binary_accuracy: 0.6711
Epoch 5/25
85/85 [==============================] - 2s 23ms/step - loss: 0.6577 - binary_accuracy: 0.6908 - val_loss: 0.6546 - val_binary_accuracy: 0.6711
Epoch 6/25
85/85 [==============================] - 2s 20ms/step - loss: 0.6347 - binary_accuracy: 0.6919 - val_loss: 0.6305 - val_binary_accuracy: 0.6711
Epoch 7/25
85/85 [==============================] - 2s 21ms/step - l

In [ ]:
bi_gru.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    validation_data = (X_test, y_test),
    epochs = 25,
)

Epoch 1/25
85/85 [==============================] - 23s 153ms/step - loss: 0.6914 - binary_accuracy: 0.5411 - val_loss: 0.6887 - val_binary_accuracy: 0.6150
Epoch 2/25
85/85 [==============================] - 6s 67ms/step - loss: 0.6833 - binary_accuracy: 0.6697 - val_loss: 0.6811 - val_binary_accuracy: 0.6785
Epoch 3/25
85/85 [==============================] - 4s 44ms/step - loss: 0.6727 - binary_accuracy: 0.6882 - val_loss: 0.6710 - val_binary_accuracy: 0.6726
Epoch 4/25
85/85 [==============================] - 4s 45ms/step - loss: 0.6591 - binary_accuracy: 0.6901 - val_loss: 0.6580 - val_binary_accuracy: 0.6711
Epoch 5/25
85/85 [==============================] - 3s 32ms/step - loss: 0.6409 - binary_accuracy: 0.6934 - val_loss: 0.6399 - val_binary_accuracy: 0.6711
Epoch 6/25
85/85 [==============================] - 2s 26ms/step - loss: 0.6176 - binary_accuracy: 0.6964 - val_loss: 0.6147 - val_binary_accuracy: 0.6711
Epoch 7/25
85/85 [==============================] - 2s 24ms/step - l

In [ ]:
stacked_bi_gru.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    validation_data = (X_test, y_test),
    epochs = 25,
)

## Model Evaluation

In [28]:
from sklearn.metrics import classification_report

class_labels = ["apresiasi", "kritik"]

y_pred = lstm.predict(X_test) > 0.5
print(classification_report(y_test, y_pred, target_names=class_labels))

11/11 [==============================] - 0s 5ms/step
              precision    recall  f1-score   support

   apresiasi       0.94      0.78      0.85       124
      kritik       0.93      0.93      0.93       240

   micro avg       0.93      0.88      0.91       364
   macro avg       0.94      0.86      0.89       364
weighted avg       0.93      0.88      0.91       364
 samples avg       0.94      0.91      0.92       364



In [29]:
y_pred = bi_lstm.predict(X_test) > 0.5
print(classification_report(y_test, y_pred, target_names=class_labels))

11/11 [==============================] - 0s 6ms/step
              precision    recall  f1-score   support

   apresiasi       0.87      0.89      0.88       124
      kritik       0.94      0.95      0.95       240

   micro avg       0.92      0.93      0.92       364
   macro avg       0.90      0.92      0.91       364
weighted avg       0.92      0.93      0.92       364
 samples avg       0.94      0.95      0.93       364



In [30]:
y_pred = stacked_bi_lstm.predict(X_test) > 0.5
print(classification_report(y_test, y_pred, target_names=class_labels))

11/11 [==============================] - 0s 11ms/step
              precision    recall  f1-score   support

   apresiasi       0.89      0.87      0.88       124
      kritik       0.93      0.95      0.94       240

   micro avg       0.92      0.93      0.92       364
   macro avg       0.91      0.91      0.91       364
weighted avg       0.92      0.93      0.92       364
 samples avg       0.94      0.95      0.93       364



In [31]:
y_pred = gru.predict(X_test) > 0.5
print(classification_report(y_test, y_pred, target_names=class_labels))

11/11 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

   apresiasi       0.89      0.78      0.83       124
      kritik       0.95      0.95      0.95       240

   micro avg       0.93      0.89      0.91       364
   macro avg       0.92      0.87      0.89       364
weighted avg       0.93      0.89      0.91       364
 samples avg       0.94      0.92      0.92       364



In [32]:
y_pred = bi_gru.predict(X_test) > 0.5
print(classification_report(y_test, y_pred, target_names=class_labels))

11/11 [==============================] - 0s 7ms/step
              precision    recall  f1-score   support

   apresiasi       0.94      0.82      0.88       124
      kritik       0.95      0.96      0.96       240

   micro avg       0.95      0.91      0.93       364
   macro avg       0.95      0.89      0.92       364
weighted avg       0.95      0.91      0.93       364
 samples avg       0.96      0.94      0.94       364



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
y_pred = stacked_bi_gru.predict(X_test) > 0.5
print(classification_report(y_test, y_pred, target_names=class_labels))

11/11 [==============================] - 0s 11ms/step
              precision    recall  f1-score   support

   apresiasi       0.83      0.89      0.86       124
      kritik       0.94      0.96      0.95       240

   micro avg       0.90      0.94      0.92       364
   macro avg       0.89      0.92      0.91       364
weighted avg       0.91      0.94      0.92       364
 samples avg       0.93      0.95      0.93       364



## Save Model

In [27]:
lstm.save("lstm-1")
bi_lstm.save("bi-lstm-1")
stacked_bi_lstm.save("stacked-bi-lstm-1")
gru.save("gru-1")
bi_gru.save("bi-gru-1")
stacked_bi_gru.save("stacked-bi-gru-1")

# BERT Model

## Mapping BERT Preprocesser and Encoder

In [12]:
map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

## Build Model Architecture

In [13]:
def build_bert_model(bert_model_name: str):
  tfhub_handle_encoder = map_name_to_handle[bert_model_name]
  tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

  text_input = tf.keras.layers.Input(shape=(), dtype="string", name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.2)(net)
  net = tf.keras.layers.Dense(2, activation='sigmoid', name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [14]:
bert_uncased_model = build_bert_model('bert_en_uncased_L-12_H-768_A-12')
small_bert_model = build_bert_model('small_bert/bert_en_uncased_L-4_H-512_A-8')
albert_model = build_bert_model('albert_en_base')
electra_model = build_bert_model('electra_base')

## Model Compiling

In [15]:
bert_uncased_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00003),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy(),
)

small_bert_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00003),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy(),
)

albert_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00003),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy(),
)

electra_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00003),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy(),
)

## Model Training

In [16]:
bert_uncased_model.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    validation_data = (X_test, y_test),
    epochs = 3,
)


Epoch 1/3
85/85 [==============================] - 102s 604ms/step - loss: 0.4883 - binary_accuracy: 0.7659 - val_loss: 0.3432 - val_binary_accuracy: 0.8481
Epoch 2/3
85/85 [==============================] - 48s 560ms/step - loss: 0.3077 - binary_accuracy: 0.8798 - val_loss: 0.2791 - val_binary_accuracy: 0.8879
Epoch 3/3
85/85 [==============================] - 45s 531ms/step - loss: 0.2394 - binary_accuracy: 0.9105 - val_loss: 0.2738 - val_binary_accuracy: 0.8923


In [17]:
small_bert_model.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    validation_data = (X_test, y_test),
    epochs = 3,
)

Epoch 1/3
85/85 [==============================] - 41s 281ms/step - loss: 0.5291 - binary_accuracy: 0.7389 - val_loss: 0.3401 - val_binary_accuracy: 0.8643
Epoch 2/3
85/85 [==============================] - 16s 192ms/step - loss: 0.3163 - binary_accuracy: 0.8724 - val_loss: 0.3260 - val_binary_accuracy: 0.8894
Epoch 3/3
85/85 [==============================] - 16s 194ms/step - loss: 0.2448 - binary_accuracy: 0.9035 - val_loss: 0.2747 - val_binary_accuracy: 0.8982


In [18]:
albert_model.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    validation_data = (X_test, y_test),
    epochs = 3,
)

Epoch 1/3
85/85 [==============================] - 77s 698ms/step - loss: 0.5715 - binary_accuracy: 0.7163 - val_loss: 0.4424 - val_binary_accuracy: 0.8230
Epoch 2/3
85/85 [==============================] - 58s 680ms/step - loss: 0.4029 - binary_accuracy: 0.8291 - val_loss: 0.3502 - val_binary_accuracy: 0.8510
Epoch 3/3
85/85 [==============================] - 56s 659ms/step - loss: 0.3082 - binary_accuracy: 0.8828 - val_loss: 0.3256 - val_binary_accuracy: 0.8879


In [19]:
electra_model.fit(
    x = X_train,
    y = y_train,
    batch_size = 16,
    validation_data = (X_test, y_test),
    epochs = 3,
)

Epoch 1/3
85/85 [==============================] - 95s 581ms/step - loss: 0.5432 - binary_accuracy: 0.7297 - val_loss: 0.3727 - val_binary_accuracy: 0.8407
Epoch 2/3
85/85 [==============================] - 46s 547ms/step - loss: 0.3110 - binary_accuracy: 0.8794 - val_loss: 0.2866 - val_binary_accuracy: 0.8938
Epoch 3/3
85/85 [==============================] - 46s 546ms/step - loss: 0.2570 - binary_accuracy: 0.8957 - val_loss: 0.2931 - val_binary_accuracy: 0.8850


## Model Evaluation

In [20]:
from sklearn.metrics import classification_report

class_labels = ["apresiasi", "kritik"]

y_pred = bert_uncased_model.predict(X_test) > 0.5
print(classification_report(y_test, y_pred, target_names=class_labels))

11/11 [==============================] - 5s 390ms/step
              precision    recall  f1-score   support

   apresiasi       0.87      0.77      0.82       124
      kritik       0.91      0.98      0.94       247

   micro avg       0.90      0.91      0.90       371
   macro avg       0.89      0.87      0.88       371
weighted avg       0.89      0.91      0.90       371
 samples avg       0.92      0.93      0.92       371



In [21]:
y_pred = small_bert_model.predict(X_test) > 0.5
print(classification_report(y_test, y_pred, target_names=class_labels))

11/11 [==============================] - 1s 97ms/step
              precision    recall  f1-score   support

   apresiasi       0.88      0.80      0.84       124
      kritik       0.92      0.96      0.94       247

   micro avg       0.91      0.91      0.91       371
   macro avg       0.90      0.88      0.89       371
weighted avg       0.90      0.91      0.91       371
 samples avg       0.93      0.93      0.92       371



In [22]:
y_pred = albert_model.predict(X_test) > 0.5
print(classification_report(y_test, y_pred, target_names=class_labels))

11/11 [==============================] - 4s 350ms/step
              precision    recall  f1-score   support

   apresiasi       0.95      0.68      0.79       124
      kritik       0.92      0.96      0.94       247

   micro avg       0.93      0.87      0.89       371
   macro avg       0.93      0.82      0.86       371
weighted avg       0.93      0.87      0.89       371
 samples avg       0.93      0.90      0.91       371



In [23]:
y_pred = electra_model.predict(X_test) > 0.5
print(classification_report(y_test, y_pred, target_names=class_labels))

11/11 [==============================] - 5s 389ms/step
              precision    recall  f1-score   support

   apresiasi       0.90      0.77      0.83       124
      kritik       0.89      0.96      0.92       247

   micro avg       0.89      0.90      0.90       371
   macro avg       0.90      0.86      0.88       371
weighted avg       0.89      0.90      0.89       371
 samples avg       0.92      0.92      0.91       371



## Save Model

In [42]:
bert_uncased_model.save("bert-1")
small_bert_model.save("small-bert-1")
albert_model.save("albert-1")
electra_model.save("electra-1")